In [4]:
import folium
import pandas as pd


In [5]:
from folium.plugins import MarkerCluster
from folium.plugins import MousePosition
from folium.features import DivIcon

combining the latitude and longitude values in our dataset

In [6]:
df= pd.read_csv("dataset_part_1.csv")
ab= pd.read_csv("combined_dataset.csv")

In [7]:
df.head()

,FlightNumber,Date,BoosterVersion,PayloadMass,Orbit,LaunchSite,Outcome,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
0,1,04-06-2010,Falcon 9,6123.547647,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1,0,B0003,-80.577366,28.561857
1,2,22-05-2012,Falcon 9,525.000000,LEO,CCSFS SLC 40,None None,1,False,False,False,NaN,1,0,B0005,-80.577366,28.561857
2,3,01-03-2013,Falcon 9,677.000000,ISS,CCSFS SLC 40,None None,1,False,False,False,NaN,1,0,B0007,-80.577366,28.561857
3,4,29-09-2013,Falcon 9,500.000000,PO,VAFB SLC 4E,False Ocean,1,False,False,False,NaN,1,0,B1003,-120.610829,34.632093
4,5,03-12-2013,Falcon 9,3170.000000,GTO,CCSFS SLC 40,None None,1,False,False,False,NaN,1,0,B1004,-80.577366,28.561857


In [8]:
ab.head()

,Flight No.,Date,Time,Version Booster,Launch site,Payload,Payload mass,Orbit,Customer,Launch Outcome,Booster landing,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial
0,1,04-06-2010,18:45,F9 v1.0B0003.1,CCAFS,Dragon Spacecraft Qualification Unit,0,LEO,SpaceX,Success\n,0,1,False,False,False,NaN,1,0,B0003
1,2,22-05-2012,07:44,F9 v1.0B0005.1,CCAFS,Dragon,525 kg,LEO,NASA,Success,0,1,False,False,False,NaN,1,0,B0005
2,3,01-03-2013,15:10,F9 v1.0B0007.1,CCAFS,SpaceX CRS-2,"4,877 kg",LEO,NASA,Success\n,0,1,False,False,False,NaN,1,0,B0007
3,4,29-09-2013,16:00,F9 v1.1B1003,VAFB,CASSIOPE,500 kg,Polar orbit,MDA,Success,0,1,False,False,False,NaN,1,0,B1003
4,5,03-12-2013,22:41,F9 v1.1,CCAFS,SES-8,"3,170 kg",GTO,SES,Success,0,1,False,False,False,NaN,1,0,B1004


In [9]:
merged_df = pd.merge(ab, df[['Date','Longitude','Latitude']], on='Date')

In [10]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 89 entries, 0 to 88
Data columns (total 21 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Flight No.       89 non-null     int64  
 1   Date             89 non-null     object 
 2   Time             89 non-null     object 
 3   Version Booster  89 non-null     object 
 4   Launch site      89 non-null     object 
 5   Payload          89 non-null     object 
 6   Payload mass     89 non-null     object 
 7   Orbit            89 non-null     object 
 8   Customer         88 non-null     object 
 9   Launch Outcome   89 non-null     object 
 10  Booster landing  89 non-null     int64  
 11  Flights          89 non-null     int64  
 12  GridFins         89 non-null     bool   
 13  Reused           89 non-null     bool   
 14  Legs             89 non-null     bool   
 15  LandingPad       63 non-null     object 
 16  Block            89 non-null     int64  
 17  ReusedCount      8

## Task 1: Mark all launch sites on a map

In [11]:
space_df = merged_df[['Launch site','Latitude','Longitude', 'Booster landing']]
launch_sites_df = space_df.groupby(['Launch site'], as_index = False).first()
launch_sites_df = launch_sites_df[['Launch site', 'Latitude','Longitude']]
launch_sites_df

,Launch site,Latitude,Longitude
0,CCAFS,28.561857,-80.577366
1,Cape Canaveral,28.561857,-80.577366
2,KSC,28.608058,-80.603956
3,VAFB,34.632093,-120.610829


We create a folium Map object, with an initial center location to be NASA Johnson Space Center at Houston, Texas.

In [12]:
nasa_cordinate = [29.559684888503615, -95.0830971930759]
site_map = folium.Map(location = nasa_cordinate, zoom_start=10)

In [13]:
circle = folium.Circle(nasa_cordinate, radius=1000, color = '#d35400', fill = True).add_child(folium.Popup('NASA Johnson Space Cente'))
#add_child is used to add popup messages in a map

In [14]:
marker = folium.map.Marker(
    nasa_cordinate,
    icon = DivIcon(
        icon_size = (20,20),
        icon_anchor= (0,0),
        html = '<div style= "font-size: 12; color: #d35400;"><b>%s</b></div>' % 'NASA JSC',
    )
)

site_map.add_child(circle)
site_map.add_child(marker)

add CCAFS Launch site on map

In [15]:
CCAFS_cordinate = [launch_sites_df.loc[0,'Latitude'],launch_sites_df.loc[0,'Longitude']]


In [16]:
circle1 = folium.Circle(CCAFS_cordinate, radius=1000, color = '#000000', fill = True).add_child(folium.Popup('CCAFS'))
#add_child is used to add popup messages in a map

marker1 = folium.map.Marker(
    CCAFS_cordinate,
    icon = DivIcon(
        icon_size = (20,20),
        icon_anchor=(0,0),
        html = '<div style = "font-size: 12; color: #000000;"><b>%s</b></div>' % 'CCAFS',
    )
)

site_map.add_child(circle1)
site_map.add_child(marker1)

Cape Canaveral launch site

In [17]:
CC_cordinate = [launch_sites_df.loc[1,'Latitude'],launch_sites_df.loc[1,'Longitude']]

In [18]:
circle2 = folium.Circle(CC_cordinate, radius=1000, color = '#000000', fill = True).add_child(folium.Popup('Cape Canaveral'))
#add_child is used to add popup messages in a map

marker2 = folium.map.Marker(
    CC_cordinate,
    icon = DivIcon(
        icon_size = (20,20),
        icon_anchor=(0,0),
        html = '<div style = "font-size: 12; color: #000000;"><b>%s</b></div>' % 'Cape Canaveral',
    )
)

site_map.add_child(circle2)
site_map.add_child(marker2)

KSC Launch Site

In [19]:
KSC_cordinate = [launch_sites_df.loc[2,'Latitude'],launch_sites_df.loc[2,'Longitude']]

circle3 = folium.Circle(CC_cordinate, radius=1000, color = '#000000', fill = True).add_child(folium.Popup('KSC'))
#add_child is used to add popup messages in a map

marker3 = folium.map.Marker(
    CC_cordinate,
    icon = DivIcon(
        icon_size = (20,20),
        icon_anchor=(0,0),
        html = '<div style = "font-size: 12; color: #000000;"><b>%s</b></div>' % 'KSC',
    )
)

site_map.add_child(circle3)
site_map.add_child(marker3)

VAFB Launch Site

In [20]:
VAFB_cordinate = [launch_sites_df.loc[3,'Latitude'],launch_sites_df.loc[3,'Longitude']]

circle4 = folium.Circle(CC_cordinate, radius=1000, color = '#000000', fill = True).add_child(folium.Popup('VAFB'))
#add_child is used to add popup messages in a map

marker4 = folium.map.Marker(
    CC_cordinate,
    icon = DivIcon(
        icon_size = (20,20),
        icon_anchor=(0,0),
        html = '<div style = "font-size: 12; color: #000000;"><b>%s</b></div>' % 'VAFB',
    )
)

site_map.add_child(circle4)
site_map.add_child(marker4)

# Task 2: Mark the success/failed launches for each site on the map


In [21]:
merged_df.tail(10)

,Flight No.,Date,Time,Version Booster,Launch site,Payload,Payload mass,Orbit,Customer,Launch Outcome,...,Flights,GridFins,Reused,Legs,LandingPad,Block,ReusedCount,Serial,Longitude,Latitude
79,80,13-06-2020,09:21,F9 B5,CCAFS,Starlink,"15,410 kg",LEO,SpaceX,Success\n,...,3,True,True,True,5e9e3032383ecb6bb234e7ca,5,5,B1059,-80.577366,28.561857
80,81,30-06-2020,20:10:46,F9 B5,CCAFS,GPS III,"4,311 kg",MEO,U.S. Space Force,Success\n,...,1,True,False,True,5e9e3033383ecbb9e534e7cc,5,12,B1060,-80.577366,28.561857
81,82,20-07-2020,21:30,F9 B5B1058.2,CCAFS,ANASIS-II,"5,000–6,000 kg",GTO,Republic of Korea Army,Success\n,...,2,True,True,True,5e9e3033383ecbb9e534e7cc,5,13,B1058,-80.577366,28.561857
82,83,18-08-2020,14:31,F9 B5B1049.6,CCAFS,Starlink,"~15,440 kg",LEO,SpaceX,Success\n,...,6,True,True,True,5e9e3032383ecb6bb234e7ca,5,9,B1049,-80.577366,28.561857
83,84,30-08-2020,23:18,F9 B5,CCAFS,SAOCOM 1B,"3,130 kg",SSO,CONAE,Success\n,...,4,True,True,True,5e9e3032383ecb267a34e7c7,5,5,B1059,-80.577366,28.561857
84,85,03-09-2020,12:46:14,F9 B5B1060.2,KSC,Starlink,"15,600 kg",LEO,SpaceX,Success\n,...,2,True,True,True,5e9e3032383ecb6bb234e7ca,5,12,B1060,-80.603956,28.608058
85,86,06-10-2020,11:29:34,F9 B5B1058.3,KSC,Starlink,"15,600 kg",LEO,SpaceX,Success\n,...,3,True,True,True,5e9e3032383ecb6bb234e7ca,5,13,B1058,-80.603956,28.608058
86,87,18-10-2020,12:25:57,F9 B5B1051.6,KSC,Starlink,"15,600 kg",LEO,SpaceX,Success\n,...,6,True,True,True,5e9e3032383ecb6bb234e7ca,5,12,B1051,-80.603956,28.608058
87,88,24-10-2020,15:31:34,F9 B5,CCAFS,Starlink,"15,600 kg",LEO,SpaceX,Success\n,...,3,True,True,True,5e9e3033383ecbb9e534e7cc,5,12,B1060,-80.577366,28.561857
88,89,05-11-2020,23:24:23,F9 B5,CCAFS,GPS III,"4,311 kg",MEO,USSF,Success\n,...,1,True,False,True,5e9e3032383ecb6bb234e7ca,5,8,B1062,-80.577366,28.561857


In [22]:
marker_cluster = MarkerCluster()

In [23]:
space_df['marker_color'] = None

<ipython-input-23-285c85cd4372>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  space_df['marker_color'] = None


In [24]:
space_df.head()

,Launch site,Latitude,Longitude,Booster landing,marker_color
0,CCAFS,28.561857,-80.577366,0,None
1,CCAFS,28.561857,-80.577366,0,None
2,CCAFS,28.561857,-80.577366,0,None
3,VAFB,34.632093,-120.610829,0,None
4,CCAFS,28.561857,-80.577366,0,None


In [25]:
def assign_color(row):
  if row['Booster landing'] == 1:
    return 'green'

  elif row['Booster landing'] == 0:
    return 'red'

  else:
    return None


In [26]:
space_df['marker_color'] = space_df.apply(assign_color, axis =1)

<ipython-input-26-37bf8b6fab2c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  space_df['marker_color'] = space_df.apply(assign_color, axis =1)


In [27]:
space_df.head(10)

,Launch site,Latitude,Longitude,Booster landing,marker_color
0,CCAFS,28.561857,-80.577366,0,red
1,CCAFS,28.561857,-80.577366,0,red
2,CCAFS,28.561857,-80.577366,0,red
3,VAFB,34.632093,-120.610829,0,red
4,CCAFS,28.561857,-80.577366,0,red
5,CCAFS,28.561857,-80.577366,0,red
6,Cape Canaveral,28.561857,-80.577366,1,green
7,Cape Canaveral,28.561857,-80.577366,1,green
8,Cape Canaveral,28.561857,-80.577366,0,red
9,Cape Canaveral,28.561857,-80.577366,0,red


In [28]:
site_map.add_child(marker_cluster)

In [29]:
marker_cluster = MarkerCluster().add_to(site_map)
for index,row in space_df.iterrows():
  marker5= folium.Marker(
      location = [row['Latitude'],row['Longitude']],
      icon = folium.Icon(color = 'white', icon_color= row['marker_color'])
  )

  marker_cluster.add_child(marker5)

site_map



# TASK 3: Calculate the distances between a launch site to its proximities

In [30]:
formatter = "function(num) {return L.Util.formatNum(num, 5);};"
mouse_position = MousePosition(
    position = 'topright',
    separator = ' Long: ',
    emplty_string = 'NaN',
    lng_first=False,
    num_digits=20,
    prefix = 'Lat:',
    lat_formatter=formatter,
    lng_formatter=formatter,
)

site_map.add_child(mouse_position)
site_map

In [31]:
from math import sin, cos, sqrt, atan2, radians

def calculate_distance(lat1, lon1, lat2, lon2):
    # approximate radius of earth in km
    R = 6373.0

    lat1 = radians(lat1)
    lon1 = radians(lon1)
    lat2 = radians(lat2)
    lon2 = radians(lon2)

    dlon = lon2 - lon1
    dlat = lat2 - lat1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    distance = R * c
    return distance

Mark down a point on the closest coastline using MousePosition and calculate the distance between the coastline point and the launch site.

### Coastline

In [32]:
#cordinates of coastline
# Lat:28.61237, Long: -80.5978
#Launch site coordinates
# Lat:28.608058	Long:-80.603956

launch_site_lat= 28.608058
launch_site_lon= -80.603956
coastline_lat = 28.61237
coastline_lon = -80.5978

distance_coastline = calculate_distance(launch_site_lat, launch_site_lon, coastline_lat, coastline_lon)
distance_coastline

0.7690174459641751

In [33]:
coast_cordinate = [coastline_lat,coastline_lon]
distance_marker = folium.Marker(
    coast_cordinate,
    icon = DivIcon(
        icon_size = (20,20),
        icon_anchor=(0,0),
        html = '<div style = "font-size:12 ; color:#d000000;"><b>%s</b></div> ' % "{:10.2f} KM".format(distance_coastline),
    )
)

site_map.add_child(distance_marker)

In [34]:
line_cordinates = [[coastline_lat,coastline_lon],[launch_site_lat,launch_site_lon]]
line = folium.PolyLine(locations = line_cordinates, weight = 2)
site_map.add_child(line)

### Railway

In [35]:
railway_lat = 28.61137
railway_lon= -80.59768

distance_railway = calculate_distance(launch_site_lat, launch_site_lon, railway_lat, railway_lon)
distance_railway

0.7150466498388548

In [36]:
railway_cordinate = [railway_lat,railway_lon]
distance_marker = folium.Marker(
    railway_cordinate,
    icon = DivIcon(
        icon_size = (20,20),
        icon_anchor=(0,0),
        html = '<div style = "font-size:12 ; color:#d000000;"><b>%s</b></div> ' % "{:10.2f} KM".format(distance_railway),
    )
)

site_map.add_child(distance_marker)

In [37]:
line_cordinates1 = [[railway_lat,railway_lon],[launch_site_lat,launch_site_lon]]
line = folium.PolyLine(locations = line_cordinates1, weight = 2)
site_map.add_child(line)